In [1]:
import pandas as pd
import numpy as np

import json
import psycopg2
import progressbar

from sqlalchemy import create_engine

## Parametros

### Conf DB

In [2]:
## Las bases de datos deben estar creadas

dbmesh = 'epi_puma_meshes'
dbname = 'epi_puma_censo_inegi_2020_new'
dbuser = 'postgres'
dbpass = 'postgres'
dbport = 5432
dbhost = '127.0.0.1'

### Conf archivo

In [3]:
filename = 'municipalities.csv'

### Conf columnas

In [4]:
covariables = ['POBTOT', 'POBFEM', 'POBMAS', 'P_0A2', 'P_0A2_F', 'P_0A2_M', 'P_3YMAS', 
               'P_3YMAS_F', 'P_3YMAS_M', 'P_5YMAS', 'P_5YMAS_F', 'P_5YMAS_M', 'P_12YMAS', 
               'P_12YMAS_F', 'P_12YMAS_M', 'P_15YMAS', 'P_15YMAS_F', 'P_15YMAS_M', 'P_18YMAS', 
               'P_18YMAS_F', 'P_18YMAS_M', 'P_3A5', 'P_3A5_F', 'P_3A5_M', 'P_6A11', 'P_6A11_F', 
               'P_6A11_M', 'P_8A14', 'P_8A14_F', 'P_8A14_M', 'P_12A14', 'P_12A14_F', 'P_12A14_M', 
               'P_15A17', 'P_15A17_F', 'P_15A17_M', 'P_18A24', 'P_18A24_F', 'P_18A24_M', 'P_15A49_F', 
               'P_60YMAS', 'P_60YMAS_F', 'P_60YMAS_M', 'REL_H_M', 'POB0_14', 'POB15_64', 'POB65_MAS', 
               'PROM_HNV', 'PNACENT', 'PNACENT_F', 'PNACENT_M', 'PNACOE', 'PNACOE_F', 'PNACOE_M', 
               'PRES2015', 'PRES2015_F', 'PRES2015_M', 'PRESOE15', 'PRESOE15_F', 'PRESOE15_M', 'P3YM_HLI', 
               'P3YM_HLI_F', 'P3YM_HLI_M', 'P3HLINHE', 'P3HLINHE_F', 'P3HLINHE_M', 'P3HLI_HE', 'P3HLI_HE_F', 
               'P3HLI_HE_M', 'P5_HLI', 'P5_HLI_NHE', 'P5_HLI_HE', 'PHOG_IND', 'POB_AFRO', 'POB_AFRO_F', 
               'POB_AFRO_M', 'PCON_DISC', 'PCDISC_MOT', 'PCDISC_VIS', 'PCDISC_LENG', 'PCDISC_AUD', 'PCDISC_MOT2', 
               'PCDISC_MEN', 'PCON_LIMI', 'PCLIM_CSB', 'PCLIM_VIS', 'PCLIM_HACO', 'PCLIM_OAUD', 'PCLIM_MOT2', 
               'PCLIM_RE_CO', 'PCLIM_PMEN', 'PSIND_LIM', 'P3A5_NOA', 'P3A5_NOA_F', 'P3A5_NOA_M', 'P6A11_NOA', 
               'P6A11_NOAF', 'P6A11_NOAM', 'P12A14NOA', 'P12A14NOAF', 'P12A14NOAM', 'P15A17A', 'P15A17A_F', 
               'P15A17A_M', 'P18A24A', 'P18A24A_F', 'P18A24A_M', 'P8A14AN', 'P8A14AN_F', 'P8A14AN_M', 'P15YM_AN', 
               'P15YM_AN_F', 'P15YM_AN_M', 'P15YM_SE', 'P15YM_SE_F', 'P15YM_SE_M', 'P15PRI_IN', 
               'P15PRI_INF', 'P15PRI_INM', 'P15PRI_CO', 'P15PRI_COF', 'P15PRI_COM', 'P15SEC_IN', 'P15SEC_INF', 
               'P15SEC_INM', 'P15SEC_CO', 'P15SEC_COF', 'P15SEC_COM', 'P18YM_PB', 'P18YM_PB_F', 'P18YM_PB_M', 
               'GRAPROES', 'GRAPROES_F', 'GRAPROES_M', 'PEA', 'PEA_F', 'PEA_M', 'PE_INAC', 'PE_INAC_F', 
               'PE_INAC_M', 'POCUPADA', 'POCUPADA_F', 'POCUPADA_M', 'PDESOCUP', 'PDESOCUP_F', 'PDESOCUP_M', 
               'PSINDER', 'PDER_SS', 'PDER_IMSS', 'PDER_ISTE', 'PDER_ISTEE', 'PAFIL_PDOM', 'PDER_SEGP', 
               'PDER_IMSSB', 'PAFIL_IPRIV', 'PAFIL_OTRAI', 'P12YM_SOLT', 'P12YM_CASA', 'P12YM_SEPA', 
               'PCATOLICA', 'PRO_CRIEVA', 'POTRAS_REL', 'PSIN_RELIG', 'TOTHOG', 'HOGJEF_F', 'HOGJEF_M', 
               'POBHOG', 'PHOGJEF_F', 'PHOGJEF_M', 'VIVTOT', 'TVIVHAB', 'TVIVPAR', 'VIVPAR_HAB', 'TVIVPARHAB', 
               'VIVPAR_DES', 'VIVPAR_UT', 'OCUPVIVPAR', 'PROM_OCUP', 'PRO_OCUP_C', 'VPH_PISODT', 'VPH_PISOTI', 
               'VPH_1DOR', 'VPH_2YMASD', 'VPH_1CUART', 'VPH_2CUART', 'VPH_3YMASC', 'VPH_C_ELEC', 'VPH_S_ELEC', 
               'VPH_AGUADV', 'VPH_AEASP', 'VPH_AGUAFV', 'VPH_TINACO', 'VPH_CISTER', 'VPH_EXCSA', 'VPH_LETR', 
               'VPH_DRENAJ', 'VPH_NODREN', 'VPH_C_SERV', 'VPH_NDEAED', 'VPH_DSADMA', 'VPH_NDACMM', 'VPH_SNBIEN', 
               'VPH_REFRI', 'VPH_LAVAD', 'VPH_HMICRO', 'VPH_AUTOM', 'VPH_MOTO', 'VPH_BICI', 'VPH_RADIO', 
               'VPH_TV', 'VPH_PC', 'VPH_TELEF', 'VPH_CEL', 'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ', 
               'VPH_SINRTV', 'VPH_SINLTC', 'VPH_SINCINT', 'VPH_SINTIC' ]

names = ['Población total', 'Población femenina', 'Población masculina', 'Población de 0 a 2 años', 
         'Población femenina de 0 a 2 años', 'Población masculina de 0 a 2 años', 'Población de 3 años y más', 
         'Población femenina de 3 años y más', 'Población masculina de 3 años y más', 'Población de 5 años y más', 
         'Población femenina de 5 años y más', 'Población masculina de 5 años y más', 'Población de 12 años y más', 
         'Población femenina de 12 años y más', 'Población masculina de 12 años y más', 'Población de 15 años y más', 
         'Población femenina de 15 años y más', 'Población masculina de 15 años y más', 'Población de 18 años y más', 
         'Población femenina de 18 años y más', 'Población masculina de 18 años y más', 'Población de 3 a 5 años', 
         'Población femenina de 3 a 5 años', 'Población masculina de 3 a 5 años', 'Población de 6 a 11 años', 
         'Población femenina de 6 a 11 años', 'Población masculina de 6 a 11 años', 'Población de 8 a 14 años', 
         'Población femenina de 8 a 14 años', 'Población masculina de 8 a 14 años', 'Población de 12 a 14 años', 
         'Población femenina de 12 a 14 años', 'Población masculina de 12 a 14 años', 'Población de 15 a 17 años', 
         'Población femenina de 15 a 17 años', 'Población masculina de 15 a 17 años', 'Población de 18 a 24 años', 
         'Población femenina de 18 a 24 años', 'Población masculina de 18 a 24 años', 'Población femenina de 15 a 49 años ', 
         'Población de 60 años y más', 'Población femenina de 60 años y más', 'Población masculina de 60 años y más', 
         'Relación hombres-mujeres', 'Población de 0 a 14 años', 'Población de 15 a 64 años', 'Población de 65 años y más', 
         'Promedio de hijas e hijos nacidos vivos', 'Población nacida en la entidad', 
         'Población femenina nacida en la entidad', 'Población masculina nacida en la entidad', 
         'Población nacida en otra entidad', 'Población femenina nacida en otra entidad', 
         'Población masculina nacida en otra entidad', 
         'Población de 5 años y más residente en la entidad en marzo de 2015', 
         'Población femenina de 5 años y más residente en la entidad en marzo de 2015', 
         'Población masculina de 5 años y más residente en la entidad en marzo de 2015', 
         'Población de 5 años y más residente en otra entidad en marzo de 2015', 
         'Población femenina de 5 años y más residente en otra entidad en marzo de 2015', 
         'Población masculina de 5 años y más residente en otra entidad en marzo de 2015', 
         'Población de 3 años y más que habla alguna lengua indígena', 
         'Población femenina de 3 años y más que habla alguna lengua indígena', 
         'Población masculina de 3 años y más que habla alguna lengua indígena', 
         'Población de 3 años y más que habla alguna lengua indígena y no habla español', 
         'Población femenina de 3 años y más que habla alguna lengua indígena y no habla español', 
         'Población masculina de 3 años y más que habla alguna lengua indígena y no habla español', 
         'Población de 3 años y más que habla alguna lengua indígena y habla español', 
         'Población femenina de 3 años y más que habla alguna lengua indígena y habla español', 
         'Población masculina de 3 años y más que habla alguna lengua indígena y habla español', 
         'Población de 5 años y más que habla alguna lengua indígena', 
         'Población de 5 años y más que habla alguna lengua indígena y no habla español', 
         'Población de 5 años y más que habla alguna lengua indígena y habla español', 
         'Población en hogares censales indígenas', 'Población que se considera afromexicana o afrodescendiente', 
         'Población femenina que se considera afromexicana o afrodescendiente', 
         'Población masculina que se considera afromexicana o afrodescendiente', 
         'Población con discapacidad', 'Población con discapacidad para caminar, subir o bajar', 
         'Población con discapacidad para ver, aun usando lentes', 
         'Población con discapacidad para hablar o comunicarse', 
         'Población con discapacidad para oír, aun usando aparato auditivo', 
         'Población con discapacidad para vestirse, bañarse o comer', 
         'Población con discapacidad para recordar o concentrarse', 'Población con limitación', 
         'Población con limitación para caminar, subir o bajar', 
         'Población con limitación para ver, aun usando lentes', 
         'Población con limitación para hablar o comunicarse', 
         'Población con limitación para oír, aun usando aparato auditivo', 
         'Población con limitación para vestirse, bañarse o comer', 
         'Población con limitación para recordar o concentrarse', 
         'Población con algún problema o condición mental', 
         'Población sin discapacidad, limitación, problema o condición mental', 
         'Población de 3 a 5 años que no asiste a la escuela', 
         'Población femenina de 3 a 5 años que no asiste a la escuela', 
         'Población masculina de 3 a 5 años que no asiste a la escuela', 
         'Población de 6 a 11 años que no asiste a la escuela', 
         'Población femenina de 6 a 11 años que no asiste a la escuela', 
         'Población masculina de 6 a 11 años que no asiste a la escuela', 
         'Población de 12 a 14 años que no asiste a la escuela', 
         'Población femenina de 12 a 14 años que no asiste a la escuela', 
         'Población masculina de 12 a 14 años que no asiste a la escuela', 
         'Población de 15 a 17 años que asiste a la escuela', 
         'Población femenina de 15 a 17 años que asiste a la escuela', 
         'Población masculina de 15 a 17 años que asiste a la escuela', 
         'Población de 18 a 24 años que asiste a la escuela', 
         'Población femenina de 18 a 24 años que asiste a la escuela', 
         'Población masculina de 18 a 24 años que asiste a la escuela', 
         'Población de 8 a 14 años que no sabe leer y escribir', 
         'Población femenina de 8 a 14 años que no sabe leer y escribir', 
         'Población masculina de 8 a 14 años que no sabe leer y escribir', 
         'Población de 15 años y más analfabeta', 'Población femenina de 15 años y más analfabeta', 
         'Población masculina de 15 años y más analfabeta', 'Población de 15 años y más sin escolaridad', 
         'Población femenina de 15 años y más sin escolaridad', 
         'Población masculina de 15 años y más sin escolaridad', 
         'Población de 15 años y más con primaria incompleta', 
         'Población femenina de 15 años y más con primaria incompleta', 
         'Población masculina de 15 años y más con primaria incompleta', 
         'Población de 15 años y más con primaria completa', 
         'Población femenina de 15 años y más con primaria completa', 
         'Población masculina de 15 años y más con primaria completa', 
         'Población de 15 años y más con secundaria incompleta', 
         'Población femenina de 15 años y más con secundaria incompleta', 
         'Población masculina de 15 años y más con secundaria incompleta', 
         'Población de 15 años y más con secundaria completa', 
         'Población femenina de 15 años y más con secundaria completa', 
         'Población masculina de 15 años y más con secundaria completa', 
         'Población de 18 años y más con educación posbásica', 
         'Población femenina de 18 años y más con educación posbásica', 
         'Población masculina de 18 años y más con educación posbásica', 'Grado promedio de escolaridad', 
         'Grado promedio de escolaridad de la población femenina', 
         'Grado promedio de escolaridad de la población masculina', 
         'Población de 12 años y más económicamente activa', 
         'Población femenina de 12 años y más económicamente activa', 
         'Población masculina de 12 años y más económicamente activa', 
         'Población de 12 años y más no económicamente activa', 
         'Población femenina de 12 años y más no económicamente activa', 
         'Población masculina de 12 años y más no económicamente activa', 
         'Población de 12 años y más ocupada', 'Población femenina de 12 años y más ocupada', 
         'Población masculina de 12 años y más ocupada', 'Población de 12 años y más desocupada', 
         'Población femenina de 12 años y más desocupada', 'Población masculina de 12 años y más desocupada', 
         'Población sin afiliación a servicios de salud', 'Población afiliada a servicios de salud', 
         'Población afiliada a servicios de salud en el IMSS', 
         'Población afiliada a servicios de salud en el ISSSTE', 
         'Población afiliada a servicios de salud en el ISSSTE estatal', 
         'Población afiliada a servicios de salud en PEMEX, Defensa o Marina', 
         'Población afiliada a servicios de salud en el Instituto de Salud para el Bienestar', 
         'Población afiliada a servicios de salud en el IMSS BIENESTAR', 
         'Población afiliada a servicios de salud en una institución privada', 
         'Población afiliada a servicios de salud en otra institución', 
         'Población de 12 años y más soltera o nunca unida', 'Población de 12 años y más casada o unida', 
         'Población de 12 años y más que estuvo casada o unida', 'Población con religión católica', 
         'Población con grupo religioso protestante/cristiano evangélico', 
         'Población con otras religiones diferentes a las anteriores', 
         'Población sin religión o sin adscripción religiosa', 'Total de hogares censales', 
         'Hogares censales con persona de referencia mujer', 'Hogares censales con persona de referencia hombre', 
         'Población en hogares censales', 'Población en hogares censales con persona de referencia mujer', 
         'Población en hogares censales con persona de referencia hombre', 'Total de viviendas', 
         'Total de viviendas habitadas', 'Total de viviendas particulares', 'Viviendas particulares habitadas', 
         'Total de viviendas particulares habitadas', 'Viviendas particulares deshabitadas', 
         'Viviendas particulares de uso temporal', 'Ocupantes en viviendas particulares habitadas', 
         'Promedio de ocupantes en viviendas particulares habitadas', 
         'Promedio de ocupantes por cuarto en viviendas particulares habitadas', 
         'Viviendas particulares habitadas con piso de material diferente de tierra', 
         'Viviendas particulares habitadas con piso de tierra', 
         'Viviendas particulares habitadas con un dormitorio', 
         'Viviendas particulares habitadas con dos dormitorios y más', 
         'Viviendas particulares habitadas con sólo un cuarto', 
         'Viviendas particulares habitadas con dos cuartos', 
         'Viviendas particulares habitadas con 3 cuartos y más', 
         'Viviendas particulares habitadas que disponen de energía eléctrica', 
         'Viviendas particulares habitadas que no disponen de energía eléctrica', 
         'Viviendas particulares habitadas que disponen de agua entubada en el ámbito de la vivienda', 
         'Viviendas particulares habitadas que disponen de agua entubada y se abastecen del servicio público de agua', 
         'Viviendas particulares habitadas que no disponen de agua entubada en el ámbito de la vivienda', 
         'Viviendas particulares habitadas que disponen de tinaco', 
         'Viviendas particulares habitadas que disponen de cisterna o aljibe', 
         'Viviendas particulares habitadas que disponen de excusado o sanitario', 
         'Viviendas particulares habitadas que disponen de letrina (pozo u hoyo)', 
         'Viviendas particulares habitadas que disponen de drenaje', 
         'Viviendas particulares habitadas que no disponen de drenaje', 
         'Viviendas particulares habitadas que disponen de energía eléctrica, agua entubada de la red pública y drenaje', 
         'Viviendas particulares habitadas que no disponen de energía eléctrica, agua entubada, ni drenaje', 
         'Viviendas particulares que disponen de drenaje y sanitario con admisión de agua', 
         'Viviendas particulares habitadas que no disponen de automóvil o camioneta, ni de motocicleta o motoneta', 
         'Viviendas particulares habitadas sin ningún bien', 
         'Viviendas particulares habitadas que disponen de refrigerador', 
         'Viviendas particulares habitadas que disponen de lavadora', 
         'Viviendas particulares habitadas que disponen de horno de microondas', 
         'Viviendas particulares habitadas que disponen de automóvil o camioneta', 
         'Viviendas particulares habitadas que disponen de motocicleta o motoneta', 
         'Viviendas particulares habitadas que disponen de bicicleta como medio de transporte', 
         'Viviendas particulares habitadas que disponen de radio', 
         'Viviendas particulares habitadas que disponen de televisor', 
         'Viviendas particulares habitadas que disponen de computadora, tablet o laptop', 
         'Viviendas particulares habitadas que disponen de línea telefónica fija', 
         'Viviendas particulares habitadas que disponen de teléfono celular', 
         'Viviendas particulares habitadas que disponen de Internet', 
         'Viviendas particulares habitadas que disponen de servicio de televisión de paga', 'Viviendas particulares habitadas que disponen de servicio de películas, música o videos de paga por Internet', 'Viviendas particulares habitadas que disponen de consola de videojuegos', 'Viviendas particulares habitadas sin radio ni televisor', 'Viviendas particulares habitadas sin línea telefónica ni teléfono celular', 'Viviendas particulares habitadas sin computadora ni Internet', 'Viviendas particulares habitadas sin tecnologías de la información y de la comunicación (TIC)']

assert len(covariables) == len(names)

map_names = {covariables[i]: names[i] for i in range(len(names))}

binning = 10
#id_cell = 'ENTIDAD'
id_cell = 'MUN'
#id_cell = 'AGEB'

columns_types = {}
columns_types[id_cell] = str

## Si se va a recrear la base de datos, o si se van a agregar mas variables
action = 'replace'
#action = 'append'


In [5]:
map_names

{'POBTOT': 'Población total',
 'POBFEM': 'Población femenina',
 'POBMAS': 'Población masculina',
 'P_0A2': 'Población de 0 a 2 años',
 'P_0A2_F': 'Población femenina de 0 a 2 años',
 'P_0A2_M': 'Población masculina de 0 a 2 años',
 'P_3YMAS': 'Población de 3 años y más',
 'P_3YMAS_F': 'Población femenina de 3 años y más',
 'P_3YMAS_M': 'Población masculina de 3 años y más',
 'P_5YMAS': 'Población de 5 años y más',
 'P_5YMAS_F': 'Población femenina de 5 años y más',
 'P_5YMAS_M': 'Población masculina de 5 años y más',
 'P_12YMAS': 'Población de 12 años y más',
 'P_12YMAS_F': 'Población femenina de 12 años y más',
 'P_12YMAS_M': 'Población masculina de 12 años y más',
 'P_15YMAS': 'Población de 15 años y más',
 'P_15YMAS_F': 'Población femenina de 15 años y más',
 'P_15YMAS_M': 'Población masculina de 15 años y más',
 'P_18YMAS': 'Población de 18 años y más',
 'P_18YMAS_F': 'Población femenina de 18 años y más',
 'P_18YMAS_M': 'Población masculina de 18 años y más',
 'P_3A5': 'Población 

### Conf meshes

In [6]:
## 'state', 'mun' o 'ageb' para malla de estados, municipios o AGEBs respectivamente

#mesh = 'state'
mesh = 'mun'
#mesh = 'ageb'

### Inicio

Calculo de deciles de variables

In [7]:
#df = pd.read_excel(filename, dtypes=columns_types)
df = pd.read_csv(filename, dtype=columns_types)

covars_list  = []

for covariable in covariables:
    
    intervals = []
    
    covar_series = df[covariable]
    covar_series = covar_series[(covar_series > -9999)]
    covar_series = covar_series[covar_series.notna()]
    covar_series = covar_series.copy()
    covar_series.sort_values(axis=0, ascending=True, inplace=True)
    
    N = covar_series.shape[0]
    
    for i in range(binning):
        inf_lim = int((N*i)/binning)
        sup_lim = int((N*(i+1))/binning)
        
        sub_covar_series = covar_series[inf_lim:sup_lim]
        sub_covar_series = sub_covar_series.copy()
        
        if covariable in ('POBTOT', 'REL_H_M', 'PROM_HNV', 'GRAPROES', 'GRAPROES_F', 'GRAPROES_M', \
                         'TOTHOG', 'VIVTOT', 'TVIVHAB', 'TVIVPAR', 'TVIVPARHAB', 'PROM_OCUP', 'PRO_OCUP_C'):
            interval = str(sub_covar_series.iloc[0]) + ':' + str(sub_covar_series.iloc[-1])
            #print(covariable, interval)
        else:
            interval = str(sub_covar_series.iloc[0]*100)[:4] + '%:' + str(sub_covar_series.iloc[-1]*100)[:4] + '%'
        
        #print(inf_lim, sup_lim, interval)
        if not interval in intervals: 
            intervals.append(interval)
            
            item = dict()
            item['name'] = map_names[covariable]
            item['interval'] = interval
            item['bin'] = i + 1
            item['code'] = covariable
            item['lim_inf'] = sub_covar_series.iloc[0]
            item['lim_sup'] = sub_covar_series.iloc[-1]
            item['mesh'] = mesh
            covars_list.append(item)
            
            if covariable in ('POBTOT', 'REL_H_M', 'PROM_HNV', 'GRAPROES', 'GRAPROES_F', 'GRAPROES_M', \
                         'TOTHOG', 'VIVTOT', 'TVIVHAB', 'TVIVPAR', 'TVIVPARHAB', 'PROM_OCUP', 'PRO_OCUP_C'):
                print(item)

{'name': 'Población total', 'interval': '81:1590', 'bin': 1, 'code': 'POBTOT', 'lim_inf': 81, 'lim_sup': 1590, 'mesh': 'mun'}
{'name': 'Población total', 'interval': '1593:3414', 'bin': 2, 'code': 'POBTOT', 'lim_inf': 1593, 'lim_sup': 3414, 'mesh': 'mun'}
{'name': 'Población total', 'interval': '3418:5690', 'bin': 3, 'code': 'POBTOT', 'lim_inf': 3418, 'lim_sup': 5690, 'mesh': 'mun'}
{'name': 'Población total', 'interval': '5723:9148', 'bin': 4, 'code': 'POBTOT', 'lim_inf': 5723, 'lim_sup': 9148, 'mesh': 'mun'}
{'name': 'Población total', 'interval': '9159:13539', 'bin': 5, 'code': 'POBTOT', 'lim_inf': 9159, 'lim_sup': 13539, 'mesh': 'mun'}
{'name': 'Población total', 'interval': '13552:19200', 'bin': 6, 'code': 'POBTOT', 'lim_inf': 13552, 'lim_sup': 19200, 'mesh': 'mun'}
{'name': 'Población total', 'interval': '19219:28130', 'bin': 7, 'code': 'POBTOT', 'lim_inf': 19219, 'lim_sup': 28130, 'mesh': 'mun'}
{'name': 'Población total', 'interval': '28155:45373', 'bin': 8, 'code': 'POBTOT', '

In [8]:
df.shape

(2469, 222)

In [10]:
covars_list

[{'name': 'Población total',
  'interval': '81:1590',
  'bin': 1,
  'code': 'POBTOT',
  'lim_inf': 81,
  'lim_sup': 1590,
  'mesh': 'mun'},
 {'name': 'Población total',
  'interval': '1593:3414',
  'bin': 2,
  'code': 'POBTOT',
  'lim_inf': 1593,
  'lim_sup': 3414,
  'mesh': 'mun'},
 {'name': 'Población total',
  'interval': '3418:5690',
  'bin': 3,
  'code': 'POBTOT',
  'lim_inf': 3418,
  'lim_sup': 5690,
  'mesh': 'mun'},
 {'name': 'Población total',
  'interval': '5723:9148',
  'bin': 4,
  'code': 'POBTOT',
  'lim_inf': 5723,
  'lim_sup': 9148,
  'mesh': 'mun'},
 {'name': 'Población total',
  'interval': '9159:13539',
  'bin': 5,
  'code': 'POBTOT',
  'lim_inf': 9159,
  'lim_sup': 13539,
  'mesh': 'mun'},
 {'name': 'Población total',
  'interval': '13552:19200',
  'bin': 6,
  'code': 'POBTOT',
  'lim_inf': 13552,
  'lim_sup': 19200,
  'mesh': 'mun'},
 {'name': 'Población total',
  'interval': '19219:28130',
  'bin': 7,
  'code': 'POBTOT',
  'lim_inf': 19219,
  'lim_sup': 28130,
  'm

### Se crea la base de datos

In [11]:
if action == 'replace':
    
    conn_string = 'postgresql://{dbuser}:{dbpass}@{dbhost}:{dbport}/{dbname}'.format(dbuser=dbuser, 
                                                                                     dbpass=dbpass,
                                                                                     dbhost=dbhost,
                                                                                     dbport=dbport,
                                                                                     dbname='postgres')

    conn = psycopg2.connect(conn_string)
    conn.autocommit = True

    try:
        cur = conn.cursor()
        cur.execute("DROP DATABASE IF EXISTS {dbname}".format(dbname=dbname))
        cur.execute("CREATE DATABASE {dbname}".format(dbname=dbname))

    except Exception as e:
        print('Error: ', str(e))
    
    cur.close()
    conn.close()

conn_string = 'postgresql://{dbuser}:{dbpass}@{dbhost}:{dbport}/{dbname}'.format(dbuser=dbuser, 
                                                                                 dbpass=dbpass,
                                                                                 dbhost=dbhost,
                                                                                 dbport=dbport,
                                                                                 dbname=dbname)

conn = psycopg2.connect(conn_string)
conn.autocommit = True

In [12]:
df_covars = pd.DataFrame.from_records(covars_list)

engine = create_engine(conn_string)
df_covars.to_sql('covariable', engine, if_exists=action, index=False, chunksize=100000)

In [13]:
covar_attributes = ['name', 'code', 'lim_inf', 'lim_sup', 'bin']

try:
    cur = conn.cursor()
    
    if action == 'replace':
        sql = """ALTER TABLE covariable ADD COLUMN id serial"""
        cur.execute(sql)

        for attr in covar_attributes:
            sql = 'CREATE INDEX idx_covariable_{attr} ON covariable("{attr}")'.format(attr=attr)
            cur.execute(sql)
        
    elif action == 'append':
        sql = """ALTER TABLE occurrence ADD COLUMN \"{mesh}\" varchar(10)""".format(mesh=id_cell)
        cur.execute(sql)
    
    cur.close()
    
except Exception as e:
        print('Error: ', str(e))

In [20]:
for covar in covariables:
    #df = pd.read_excel(filename, dtypes=columns_types)
    print(covar)
    df = pd.read_csv(filename, dtype=columns_types)
    N = df.shape[0]
    
    sql = """SELECT id FROM covariable WHERE code='{code}' and mesh = '{mesh}' 
                AND bin=10 ORDER BY id DESC
            """.format(code=covar, mesh=mesh)
    cur = conn.cursor()
    cur.execute(sql)
    results = cur.fetchall()
    if len(results) > 0:
        id_covariable = results[0][0]
    else:
        print('BIN 10: ', sql)
        id_covariable = 0
        
    df['covariable_id'] = pd.Series(data=[id_covariable for i in range(N)])
    
    for i in range(N):
        val = df.iloc[i][covar]
        if str(val) != 'nan':
            sql = """SELECT id, lim_inf, lim_sup FROM covariable WHERE code='{code}' and mesh = '{mesh}' 
                        AND ((lim_inf <= {val} AND {val} <= lim_sup))
                    """.format(code=covar, val=val, binning=binning, mesh=mesh)
            cur = conn.cursor()
            cur.execute(sql)
            results = cur.fetchall()
            if len(results) > 0:
                id_covariable = results[0][0]
                df.at[i, 'covariable_id'] = id_covariable
                #print(i, val, id_covariable)
            else:
                #print(sql)
                pass
        else:
            df.at[i, 'covariable_id'] = -9999
    
    #df = df.drop(columns=covariables, axis=1)
    
    engine = create_engine(conn_string)
    df.to_sql('occurrence', engine, if_exists='append', index=False, chunksize=100000)

    cur.close()

POBTOT
POBFEM
POBMAS
P_0A2
P_0A2_F
P_0A2_M
P_3YMAS
P_3YMAS_F
P_3YMAS_M
P_5YMAS
P_5YMAS_F
P_5YMAS_M
P_12YMAS
P_12YMAS_F
P_12YMAS_M
P_15YMAS
P_15YMAS_F
P_15YMAS_M
P_18YMAS
P_18YMAS_F
P_18YMAS_M
P_3A5
P_3A5_F
P_3A5_M
P_6A11
P_6A11_F
P_6A11_M
P_8A14
P_8A14_F
P_8A14_M
P_12A14
P_12A14_F
P_12A14_M
P_15A17
P_15A17_F
P_15A17_M
P_18A24
P_18A24_F
P_18A24_M
P_15A49_F
P_60YMAS
P_60YMAS_F
P_60YMAS_M
REL_H_M
POB0_14
POB15_64
POB65_MAS
PROM_HNV
PNACENT
PNACENT_F
PNACENT_M
PNACOE
PNACOE_F
PNACOE_M
PRES2015
PRES2015_F
PRES2015_M
PRESOE15
PRESOE15_F
PRESOE15_M
P3YM_HLI
P3YM_HLI_F
P3YM_HLI_M
P3HLINHE
P3HLINHE_F
P3HLINHE_M
P3HLI_HE
P3HLI_HE_F
P3HLI_HE_M
P5_HLI
P5_HLI_NHE
P5_HLI_HE
PHOG_IND
POB_AFRO
POB_AFRO_F
POB_AFRO_M
PCON_DISC
PCDISC_MOT
PCDISC_VIS
PCDISC_LENG
PCDISC_AUD
PCDISC_MOT2
PCDISC_MEN
PCON_LIMI
PCLIM_CSB
PCLIM_VIS
PCLIM_HACO
PCLIM_OAUD
PCLIM_MOT2
PCLIM_RE_CO
PCLIM_PMEN
PSIND_LIM
P3A5_NOA
P3A5_NOA_F
P3A5_NOA_M
P6A11_NOA
P6A11_NOAF
P6A11_NOAM
P12A14NOA
P12A14NOAF
P12A14NOAM
P15A17A
P15A17A_F
P15A

In [21]:
try:
    cur = conn.cursor()
    
    if mesh != 'state':
        sql = "ALTER TABLE occurrence ADD COLUMN gridid_state varchar(10)".format(id_cell=id_cell)
    else:
        sql = "ALTER TABLE occurrence RENAME COLUMN \"{id_cell}\" TO gridid_state".format(id_cell=id_cell)
    cur.execute(sql)
    
    if mesh != 'ageb':
        sql = "ALTER TABLE occurrence ADD COLUMN gridid_ageb varchar(10)".format(id_cell=id_cell)
    else:
        sql = "ALTER TABLE occurrence RENAME COLUMN \"{id_cell}\" TO gridid_ageb".format(id_cell=id_cell)
    cur.execute(sql)
    
    if mesh != 'mun':
        sql = "ALTER TABLE occurrence ADD COLUMN gridid_mun varchar(10)".format(id_cell=id_cell)
    else:
        sql = "ALTER TABLE occurrence RENAME COLUMN \"{id_cell}\" TO gridid_mun".format(id_cell=id_cell)
    cur.execute(sql)
    
    sql = "CREATE INDEX idx_occurrence_gridid_{grid} ON occurrence(gridid_{grid})".format(grid=mesh)
    cur.execute(sql)
    
    if action == 'replace':
        sql = "CREATE INDEX idx_occurrence_gridid_id_covariable ON occurrence(covariable_id)"
        cur.execute(sql)
    
    cur.close()
    
except Exception as e:
    print('Error: ', str(e))

In [22]:
try:
    cur = conn.cursor()
    sql = "ALTER TABLE occurrence ADD COLUMN id serial"
    cur.execute(sql)
    
    sql = "ALTER TABLE occurrence ADD COLUMN date_occurrence timestamp DEFAULT '2020-01-01'"
    cur.execute(sql)
    
    sql = "CREATE INDEX idx_occurrence_date_occurrence ON occurrence(date_occurrence)"
    cur.execute(sql)
    
    cur.close()
except Exception as e:
    print('Error: ', str(e))

In [23]:
try:
    cur = conn.cursor()
    if action == 'replace':
        sql = "CREATE INDEX idx_occurrence_id_covariable ON occurrence(covariable_id)"
        cur.execute(sql)
    
    sql = "ALTER TABLE covariable ADD COLUMN cells_state varchar(10)[]"
    cur.execute(sql)
    
    sql = "ALTER TABLE covariable ADD COLUMN cells_mun varchar(10)[]"
    cur.execute(sql)
    
    sql = "ALTER TABLE covariable ADD COLUMN cells_ageb varchar(10)[]"
    cur.execute(sql)

    sql = "CREATE INDEX idx_covariable_cells_state ON covariable USING GIN(cells_state)"
    cur.execute(sql)
    
    sql = "CREATE INDEX idx_covariable_cells_mun ON covariable USING GIN(cells_mun)"
    cur.execute(sql)
    
    sql = "CREATE INDEX idx_covariable_cells_ageb ON covariable USING GIN(cells_ageb)"
    cur.execute(sql)
    
    cur.close()
except Exception as e:
    print('Error: ', str(e))

In [24]:
cur = conn.cursor()
cur.execute("SELECT id FROM covariable WHERE mesh='state' and cells_state is null")
covar_ids = cur.fetchall()
covar_ids = [it[0] for it in covar_ids]

for i in covar_ids:
    cur = conn.cursor()
    cur.execute("UPDATE covariable SET cells_state = array(SELECT DISTINCT gridid_state FROM occurrence WHERE not gridid_state is null AND covariable_id =  {id_covariable})::varchar(10)[] WHERE covariable.id = {id_covariable}". format(id_covariable=i))

cur.close()

In [25]:
cur = conn.cursor()
cur.execute("SELECT id FROM covariable WHERE mesh='mun' and cells_mun is null")
covar_ids = cur.fetchall()
covar_ids = [it[0] for it in covar_ids]

for i in covar_ids:
    cur = conn.cursor()
    cur.execute("UPDATE covariable SET cells_mun = array(SELECT DISTINCT gridid_mun FROM occurrence WHERE not gridid_mun is null AND covariable_id =  {id_covariable})::varchar(10)[] WHERE covariable.id = {id_covariable}". format(id_covariable=i))

cur.close()

In [26]:
cur = conn.cursor()
cur.execute("SELECT id FROM covariable WHERE mesh='ageb' and cells_ageb is null")
covar_ids = cur.fetchall()
covar_ids = [it[0] for it in covar_ids]

for i in covar_ids:
    cur = conn.cursor()
    cur.execute("UPDATE covariable SET cells_ageb = array(SELECT DISTINCT gridid_ageb FROM occurrence WHERE not gridid_ageb is null AND covariable_id =  {id_covariable})::varchar(10)[] WHERE covariable.id = {id_covariable}". format(id_covariable=i))

cur.close()

In [27]:
conn.close()